<a href="https://www.kaggle.com/code/maita27/digit-recognizer?scriptVersionId=253840084" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [3]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [16]:
train.isnull().sum()

label       0
pixel0      0
pixel1      0
pixel2      0
pixel3      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Length: 785, dtype: int64

In [6]:
pixel_df = train.drop(columns = ['label'], inplace = False)

In [26]:
pixel_df

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#function to get all pixel values for a specific row
def get_row_pixels(df, row):
    pixel_values = []
    for column_nr in range(0, 784):
        pixel_values.append(df[f'pixel{column_nr}'][row])
    return pixel_values

#function to make a list of lists for every row in the df
def list_of_lists(df):
    processed_img_list = []
    for i in range(0, 42000):
        processed_img_list.append(get_row_pixels(df, i))
                                  
print(list_of_lists(pixel_df))

# 

Version2:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn.functional as F

In [ ]:
df_train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
df_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
print(df_train.shape)

In [ ]:
df_train.head()

In [ ]:
y_train = df_train['label'].values.astype(np.int64)
x_train = df_train.drop(columns = ['label']).values.astype(np.float32)

In [ ]:
plt.imshow(x_train[3].reshape(28, 28), cmap = 'gray')

In [ ]:
class DigitDataset(Dataset):
    def __init__(self, x_df, y_df):
        self.x = x_df.astype(np.float32)
        self.y = y_df.astype(np.int64)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        #get image and label
        image = self.x[idx].reshape(1, 28, 28)
        label = self.y[idx]
        return torch.tensor(image), torch.tensor(label)

In [ ]:
train_dataset = DigitDataset(x_train, y_train)

In [ ]:
#wrap dataset in a dataloader

train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
#batch_size = how many sample the model looks at before updating its weights
#shuffle = mixing data randomply before each epochs

In [ ]:
#build a model

class SimpleNN(nn.Module): #SimpleNN inherits nn.Module
    def __init__(self):
        super().__init__() #inheritS the __init__ from nn.Nodule

        #nn.Linear(in_features, out_featres)
        
        self.fc1 = nn.Linear(28*28, 128) #input layer to hidden layer, 784 -> 128, so each of the 784 inputs connect to each 128 neurons
        self.fc2 = nn.Linear(128, 64) 
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1) #flatten the 1x28x28 images into 784 vector
        x = F.relu(self.fc1(x)) # relu activation
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # raw scores (logits)
        return x

In [ ]:
#instatiating model

model = SimpleNN()
criterion = nn.CrossEntropyLoss() #for multi-class classificatoin
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
#training loop

num_epochs = 5 #num of full passthroughs of training dataset

for epoch in range(num_epochs):
    model.train() #set model in training mode
    running_loss = 0

    for images, labels in train_loader:
        optimizer.zero_grad() #zero the gradients
        outputs = model(images) #forward pass
        loss = criterion(outputs, labels) #compute loss
        loss.backward() #backpropagation
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}')

In [ ]:
x_test = df_test.values.astype(np.float32)
x_test_tensor = torch.tensor(x_test).view(-1, 1, 28, 28)

In [ ]:
model.eval()
predictions = []

with torch.no_grad():
    for i in range(0, len(x_test_tensor), 64): #64 = batch size
        batch = x_test_tensor[i:i+64]
        outputs = model(batch)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

In [ ]:
sub = pd.DataFrame({
    'ImageId':list(range(1, len(predictions) +1 )),
    'Label':predictions
})

sub.to_csv('submission.csv', index = False)